In [ ]:
import pandas as pd
import numpy as np

class LightCurveProcessor:
    """
    Lớp chịu trách nhiệm tải, làm sạch và hiệu chỉnh vật lý cho dữ liệu lightcurve LSST.
    """
    
    # Hệ số tắt dần Schlafly & Finkbeiner (2011) cho LSST filters (Rv=3.1)
    EXTINCTION_COEFFS = {
        'u': 4.239, 'g': 3.303, 'r': 2.285, 
        'i': 1.698, 'z': 1.263, 'y': 1.086
    }

    def __init__(self, metadata_path):
        """
        Khởi tạo với đường dẫn đến file metadata chứa thông tin E(B-V).
        """
        self.metadata = pd.read_csv(metadata_path)
        # Tạo index bằng object_id để tra cứu nhanh khi merge
        if 'object_id' in self.metadata.columns:
            self.metadata.set_index('object_id', inplace=True)
        
    def correct_extinction(self, df):
        """
        Áp dụng hiệu chỉnh tắt dần thiên hà cho thông lượng.
        Công thức: F_corr = F_obs * 10^(0.4 * A_lambda)
        với A_lambda = R_lambda * E(B-V)
        """
        # 1. Join với metadata để lấy EBV
        if 'EBV' not in df.columns:
            # Sử dụng map hoặc merge. Vì metadata đã set index là object_id,
            # ta dùng merge với right_index=True để tối ưu tốc độ.
            # Chỉ lấy cột EBV để tránh trùng lặp dữ liệu không cần thiết
            df = df.merge(
                self.metadata[['EBV']], 
                left_on='object_id', 
                right_index=True, 
                how='left'
            )
            
        # 2. Ánh xạ hệ số R_lambda dựa trên bộ lọc (SỬA LỖI DÒNG 39)
        # Lưu vào cột mới thay vì đè lên df
        df['R_lambda'] = df['Filter'].map(self.EXTINCTION_COEFFS)
        
        # 3. Tính toán độ tắt dần A_lambda (SỬA LỖI DÒNG 42)
        # A_lambda = R_lambda * E(B-V)
        df['A_lambda'] = df['R_lambda'] * df['EBV']
        
        # 4. Hệ số hiệu chỉnh thông lượng
        correction_factor = np.power(10, 0.4 * df['A_lambda'])
        
        # 5. Áp dụng hiệu chỉnh cho cả Flux và Flux_err
        df['Flux_corr'] = df['Flux'] * correction_factor
        df['Flux_err_corr'] = df['Flux_err'] * correction_factor
        
        # 6. Làm sạch các cột tạm thời (SỬA LỖI DÒNG 51)
        # Xử lý trường hợp EBV bị NaN (do không tìm thấy object_id)
        df.dropna(subset=['Flux_corr'], inplace=True)
        
        return df.drop(columns=['R_lambda', 'A_lambda', 'EBV'])

    def load_and_process(self, lc_path):
        """
        Pipeline tải và xử lý một file csv lightcurve.
        """
        print(f"Đang tải dữ liệu từ {lc_path}...")
        raw_df = pd.read_csv(lc_path)
        
        # Pipeline xử lý
        try:
            processed_df = self.correct_extinction(raw_df)
            print("Hoàn tất hiệu chỉnh extinction.")
            return processed_df
        except Exception as e:
            print(f"Lỗi trong quá trình xử lý: {e}")
            return raw_df

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel
import numpy as np

class GPRInterpolator:
    """
    Thực hiện hồi quy GP để nội suy và tăng cường dữ liệu lightcurve.
    Sử dụng Kernel Matern 3/2 phù hợp cho vật lý thiên văn.
    """
    def __init__(self, time_grid_length=100, time_range=(-50, 150)):
        # Định nghĩa lưới thời gian chuẩn hóa: từ -50 đến +150 ngày so với đỉnh
        self.time_grid = np.linspace(time_range, time_range, time_grid_length)
        
        # Cấu hình Kernel:
        # ConstantKernel: Điều chỉnh biên độ tổng thể
        # Matern(nu=1.5): Điều chỉnh độ trơn (smoothness)
        # length_scale_bounds: Giới hạn thang thời gian biến đổi (1 ngày đến 100 ngày)
        self.kernel = ConstantKernel(1.0, (1e-3, 1e3)) * \
                      Matern(length_scale=20, length_scale_bounds=(1, 200), nu=1.5)

    def fit_predict(self, times, fluxes, errors, augment=0):
        """
        Khớp mô hình GP và trả về lưới nội suy.
        
        Args:
            times (array): Thời gian quan sát (đã chuẩn hóa t_peak=0).
            fluxes (array): Thông lượng đã hiệu chỉnh.
            errors (array): Sai số thông lượng đã hiệu chỉnh.
            augment (int): Số lượng mẫu tăng cường cần tạo ra từ phân phối hậu nghiệm.
            
        Returns:
            np.array: Ma trận shape (1 + augment, time_grid_length)
        """
        # Sklearn yêu cầu input shape (N, 1)
        X = times.reshape(-1, 1)
        y = fluxes
        
        # alpha nhận giá trị phương sai (error^2) để xử lý nhiễu dị phương sai
        alpha = errors ** 2
        
        # Khởi tạo GPR với n_restarts_optimizer để tránh tối ưu cục bộ
        gpr = GaussianProcessRegressor(kernel=self.kernel, alpha=alpha, 
                                       n_restarts_optimizer=3, normalize_y=True)
        
        try:
            gpr.fit(X, y)
        except Exception as e:
            # Fallback an toàn nếu GPR thất bại (hiếm gặp)
            print(f"GPR fit failed: {e}")
            return np.zeros((augment + 1, len(self.time_grid)))

        X_pred = self.time_grid.reshape(-1, 1)
        
        # 1. Dự đoán giá trị trung bình (Mean prediction) - Đây là mẫu chính
        mu = gpr.predict(X_pred)
        results = [mu]
        
        # 2. Tăng cường dữ liệu (Augmentation): Sample từ hậu nghiệm
        if augment > 0:
            # sample_y trả về shape (n_targets, n_samples), cần transpose
            samples = gpr.sample_y(X_pred, n_samples=augment, random_state=42)
            for i in range(augment):
                results.append(samples[:, i])
                
        return np.array(results)

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from scipy.stats import skew, kurtosis, linregress

class PhysicsFeatureExtractor:
    def __init__(self):
        self.bands = ['u', 'g', 'r', 'i', 'z', 'y']
        self.n_feat_per_band = 26
        self.n_color_feat = 10
        self.total_features = (6 * 26) + 10 # = 166

    @staticmethod
    def bazin(t, A, B, t0, t_fall, t_rise):
        with np.errstate(over='ignore', invalid='ignore'):
            exp_fall = np.exp(-(t - t0) / np.clip(t_fall, 1e-3, 500))
            exp_rise = np.exp(-(t - t0) / np.clip(t_rise, 1e-3, 200))
            val = A * (exp_fall / (1 + exp_rise)) + B
        return np.nan_to_num(val)

    def fit_bazin(self, times, fluxes):
        if len(fluxes) <= 5: return [0]*6 # Fix: Phải > 5 mới tính được chi2
        try:
            p0 = [np.max(fluxes)-np.min(fluxes), np.min(fluxes), times[np.argmax(fluxes)], 30, 15]
            bounds = ([0, -np.inf, times.min()-20, 0.1, 0.1], [np.inf, np.inf, times.max()+20, 500, 200])
            popt, _ = curve_fit(self.bazin, times, fluxes, p0=p0, bounds=bounds, maxfev=800)
            residuals = fluxes - self.bazin(times, *popt)
            # Fix: Tránh chia cho 0 hoặc số âm
            chi2 = np.sum(residuals**2) / (len(fluxes) - 5)
            return list(popt) + [chi2]
        except:
            return [0]*6

    def calculate_stats(self, times, fluxes, errors):
        if len(fluxes) == 0: return [0] * 20
        
        # Tránh std = 0
        f_std = np.std(fluxes)
        f_mean = np.mean(fluxes)
        
        # Các chỉ số cơ bản
        res = [
            np.max(fluxes), np.min(fluxes), f_mean, f_std,
            skew(fluxes) if len(fluxes) > 2 else 0,
            kurtosis(fluxes) if len(fluxes) > 2 else 0,
            (np.max(fluxes) - np.min(fluxes)) / 2,
            np.median(fluxes)
        ]
        
        # Percentiles
        q5, q25, q75, q95 = np.percentile(fluxes, [5, 25, 75, 95])
        res.extend([q5, q25, q75, q95, q75 - q25])
        
        # Ratios (Dùng np.divide để an toàn)
        res.append(np.mean(fluxes**2))
        res.append(np.mean(np.divide(fluxes, errors + 1e-6)))
        res.append(np.sum(fluxes > f_mean) / len(fluxes))
        
        # Stetson K
        delta = np.divide(fluxes - f_mean, errors + 1e-9)
        res.append(np.sum(np.abs(delta)) / len(fluxes) * np.sqrt(1.0/len(fluxes)))
        
        # Slope
        try:
            slope, intercept, _, _, _ = linregress(times, fluxes)
            res.extend([slope, intercept])
        except:
            res.extend([0, 0])
            
        res.append(len(fluxes))
        return res

    def extract_features(self, df_object):
        features = []
        band_max_flux = {b: 0 for b in self.bands}
        band_max_time = {b: 0 for b in self.bands}

        for band in self.bands:
            band_data = df_object[df_object['Filter'] == band]
            if len(band_data) < 3:
                features.extend([0] * self.n_feat_per_band)
            else:
                times, fluxes, errors = band_data['Time'].values, band_data['Flux_corr'].values, band_data['Flux_err_corr'].values
                stats = self.calculate_stats(times, fluxes, errors)
                bazin = self.fit_bazin(times, fluxes) if band in ['g', 'r', 'i'] else [0]*6
                features.extend(stats + bazin)
                band_max_flux[band] = stats[0]
                band_max_time[band] = times[np.argmax(fluxes)]

        # Color features
        for b1, b2 in [('u', 'g'), ('g', 'r'), ('r', 'i'), ('i', 'z'), ('z', 'y')]:
            if band_max_flux[b1] > 0 and band_max_flux[b2] > 0:
                color = -2.5 * np.log10(band_max_flux[b1] / band_max_flux[b2])
                lag = band_max_time[b1] - band_max_time[b2]
            else:
                color, lag = 0, 0
            features.extend([color, lag])
            
        return np.array(features)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class Conv1DAutoencoder(nn.Module):
    """
    Mạng nơ-ron tích chập 1 chiều Autoencoder để trích xuất đặc trưng hình thái học.
    """
    def __init__(self, input_len=100, num_channels=6, latent_dim=32):
        super(Conv1DAutoencoder, self).__init__()
        
        # --- Encoder ---
        # Input: (Batch, 6, 100)
        self.encoder = nn.Sequential(
            # Layer 1: 100 -> 50
            nn.Conv1d(num_channels, 16, kernel_size=3, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
            
            # Layer 2: 50 -> 25
            nn.Conv1d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),
            
            # Layer 3: 25 -> 5
            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(5) 
        )
        
        # Flatten: 64 channels * 5 length = 320 features
        self.flatten_dim = 64 * 5
        
        # Latent Vector (Bottleneck)
        self.fc_enc = nn.Linear(self.flatten_dim, latent_dim)
        
        # --- Decoder ---
        self.fc_dec = nn.Linear(latent_dim, self.flatten_dim)
        
        self.decoder = nn.Sequential(
            # Unflatten -> (Batch, 64, 5)
            # Layer 1: 5 -> 25
            nn.ConvTranspose1d(64, 32, kernel_size=5, stride=5),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            
            # Layer 2: 25 -> 50
            nn.ConvTranspose1d(32, 16, kernel_size=2, stride=2),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            
            # Layer 3: 50 -> 100
            nn.ConvTranspose1d(16, num_channels, kernel_size=2, stride=2),
            # Output không dùng hàm kích hoạt để hồi quy giá trị thực
        )

    def forward(self, x):
        # Encode
        x = self.encoder(x)
        x = x.view(x.size(0), -1) # Flatten
        latent = self.fc_enc(x)
        
        # Decode
        x = self.fc_dec(latent)
        x = x.view(x.size(0), 64, 5) # Reshape
        reconstruction = self.decoder(x)
        
        return reconstruction, latent

    def get_latent(self, x):
        """Trích xuất véc-tơ tiềm ẩn cho mục đích phân loại."""
        self.eval()
        with torch.no_grad():
            x = self.encoder(x)
            x = x.view(x.size(0), -1)
            latent = self.fc_enc(x)
        return latent.cpu().numpy()

In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

class HybridEnsemble:
    """
    Mô hình Stacking kết hợp XGBoost, LightGBM, CatBoost.
    Meta-learner: Logistic Regression.
    """
    def __init__(self, scale_pos_weight=1.0):
        # Cấu hình các base models với tham số xử lý mất cân bằng
        self.xgb = XGBClassifier(
            n_estimators=500, learning_rate=0.05, max_depth=6,
            scale_pos_weight=scale_pos_weight,
            eval_metric='logloss', use_label_encoder=False
        )
        self.lgbm = LGBMClassifier(
            n_estimators=500, learning_rate=0.05,
            scale_pos_weight=scale_pos_weight,
            verbose=-1
        )
        self.cat = CatBoostClassifier(
            iterations=500, learning_rate=0.05, 
            scale_pos_weight=scale_pos_weight,
            verbose=0, allow_writing_files=False
        )
        
        self.meta = LogisticRegression()
        self.scaler = StandardScaler()

    def fit(self, X_phys, X_dl, y):
        """
        Huấn luyện Stacking theo quy trình Cross-Validation Out-of-Fold.
        """
        # 1. Kết hợp đặc trưng và chuẩn hóa
        X_combined = np.hstack([X_phys, X_dl])
        X_scaled = self.scaler.fit_transform(X_combined)
        
        # 2. Tạo Out-of-Fold Predictions cho Meta-learner
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        meta_features = np.zeros((len(y), 3)) # 3 models
        
        print("Đang huấn luyện Base Learners (Level 0)...")
        for fold, (train_idx, val_idx) in enumerate(skf.split(X_scaled, y)):
            X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
            
            # Fit từng model
            self.xgb.fit(X_train, y_train)
            self.lgbm.fit(X_train, y_train)
            self.cat.fit(X_train, y_train)
            
            # Predict proba cho tập val
            meta_features[val_idx, 0] = self.xgb.predict_proba(X_val)[:, 1]
            meta_features[val_idx, 1] = self.lgbm.predict_proba(X_val)[:, 1]
            meta_features[val_idx, 2] = self.cat.predict_proba(X_val)[:, 1]
            
        # 3. Retrain base models trên toàn bộ dữ liệu
        print("Retraining Base Learners trên toàn bộ dữ liệu...")
        self.xgb.fit(X_scaled, y)
        self.lgbm.fit(X_scaled, y)
        self.cat.fit(X_scaled, y)
        
        # 4. Train Meta-learner trên meta_features
        print("Huấn luyện Meta Learner (Level 1)...")
        self.meta.fit(meta_features, y)

    def predict_proba(self, X_phys, X_dl):
        X_combined = np.hstack([X_phys, X_dl])
        X_scaled = self.scaler.transform(X_combined)
        
        # Lấy dự đoán từ base models
        p1 = self.xgb.predict_proba(X_scaled)[:, 1]
        p2 = self.lgbm.predict_proba(X_scaled)[:, 1]
        p3 = self.cat.predict_proba(X_scaled)[:, 1]
        
        # Meta features cho test set
        meta_features = np.column_stack([p1, p2, p3])
        
        # Dự đoán cuối cùng
        return self.meta.predict_proba(meta_features)[:, 1]

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os
import gc
import warnings
from joblib import Parallel, delayed
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel
from sklearn.exceptions import ConvergenceWarning

# --- HÀM XỬ LÝ 1 OBJECT (ĐÃ FIX CHẶN WARNING TẠI LUỒNG CON) ---
def process_single_object(obj_id, obj_data, target, n_augment, phys_extractor):
    import warnings
    from sklearn.exceptions import ConvergenceWarning
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning) # Chặn warning chia 0
    
    try:
        if obj_data.empty: return None
        
        # 1. Trích xuất đặc trưng vật lý (166 đặc trưng)
        try:
            phys_feats = phys_extractor.extract_features(obj_data)
            if len(phys_feats) != 166: # Kiểm tra chéo độ dài
                phys_feats = np.zeros(166)
        except:
            phys_feats = np.zeros(166)

        # 2. Xử lý Gaussian Process (DL Features)
        t_peak_global = obj_data.loc[obj_data['Flux_corr'].idxmax(), 'Time']
        time_grid = np.linspace(-50, 150, 100)
        obj_grids = []
        expected_shape = (n_augment + 1, 100)
        
        kernel = ConstantKernel(1.0) * Matern(length_scale=20, nu=1.5)

        for band in ['u', 'g', 'r', 'i', 'z', 'y']:
            band_dat = obj_data[obj_data['Filter'] == band]
            if len(band_dat) < 3:
                interp = np.zeros(expected_shape)
            else:
                X = (band_dat['Time'].values - t_peak_global).reshape(-1, 1)
                y = band_dat['Flux_corr'].values
                alpha = band_dat['Flux_err_corr'].values**2 + 1e-4
                gpr = GaussianProcessRegressor(kernel=kernel, alpha=alpha, normalize_y=True)
                try:
                    gpr.fit(X, y)
                    mu = gpr.predict(time_grid.reshape(-1, 1))
                    results = [mu]
                    if n_augment > 0:
                        samples = gpr.sample_y(time_grid.reshape(-1, 1), n_samples=n_augment)
                        for k in range(n_augment): results.append(samples[:, k])
                    interp = np.array(results)
                except:
                    interp = np.zeros(expected_shape)
            
            # Chuẩn hóa shape (1 + n_aug, 100)
            if interp.shape != expected_shape:
                interp = np.zeros(expected_shape)
            obj_grids.append(interp)

        # Gộp các băng: (1 + n_aug, 6, 100)
        stacked_grids = np.stack(obj_grids, axis=1)

        final_res = []
        for i in range(n_augment + 1):
            final_res.append({
                'dl_feat': stacked_grids[i], 
                'phys_feat': phys_feats, 
                'label': target, 
                'obj_id': obj_id
            })
        return final_res
    except Exception as e:
        return None

def main():
    print("=== BẮT ĐẦU PIPELINE TỐI ƯU (SIÊU IM LẶNG) ===")
    
    BASE_DIR = '/kaggle/input/data-for-black-hole'
    TRAIN_META_PATH = os.path.join(BASE_DIR, 'train_log.csv')
    TEST_META_PATH = os.path.join(BASE_DIR, 'test_log.csv')
    
    if not os.path.exists(TRAIN_META_PATH): raise FileNotFoundError("Thiếu metadata!")
    
    split_dirs = sorted([d for d in os.listdir(BASE_DIR) if d.startswith('split_')])
    print(f"-> Tìm thấy {len(split_dirs)} splits.")

    train_processor = LightCurveProcessor(TRAIN_META_PATH)
    test_processor = LightCurveProcessor(TEST_META_PATH) if os.path.exists(TEST_META_PATH) else None
    train_meta = pd.read_csv(TRAIN_META_PATH)
    physics_extractor = PhysicsFeatureExtractor()
    
    FINAL_X_DL, FINAL_X_PHYS, FINAL_Y = [], [], []

    # PHẦN 1: TRAIN
    print(f"\n[PHẦN 1] Xử lý dữ liệu TRAIN (Dùng 100% CPU)...")
    for split_name in split_dirs:
        LC_PATH = os.path.join(BASE_DIR, split_name, 'train_full_lightcurves.csv')
        if not os.path.exists(LC_PATH): continue
        
        print(f"  >> Đang chạy Split: {split_name}...")
        train_lc = train_processor.load_and_process(LC_PATH)
        
        # Đồng bộ cột Time
        cols = train_lc.columns.tolist()
        time_col = next((c for c in cols if 'mjd' in c.lower() or 'time' in c.lower()), 'Time')
        train_lc.rename(columns={time_col: 'Time'}, inplace=True)
        
        unique_objs = train_lc['object_id'].unique()
        parallel_inputs = []
        for obj_id in unique_objs:
            target = train_meta[train_meta['object_id'] == obj_id]['target'].values
            if len(target) > 0:
                obj_data = train_lc[train_lc['object_id'] == obj_id].copy()
                parallel_inputs.append((obj_id, obj_data, target[0], 5 if target[0] == 1 else 0))
        
        # THỰC THI SONG SONG (Dòng quan trọng)
        # N_JOBS=-1: Tận dụng mọi nhân CPU
        batch_results = Parallel(n_jobs=-1)(
            delayed(process_single_object)(pid, pdata, ptarget, paug, physics_extractor) 
            for pid, pdata, ptarget, paug in parallel_inputs
        )
        
        for res in batch_results:
            if res:
                for s in res:
                    FINAL_X_DL.append(s['dl_feat'])
                    FINAL_X_PHYS.append(s['phys_feat'])
                    FINAL_Y.append(s['label'])
        
        del train_lc, batch_results; gc.collect()

    # PHẦN 2 & 3: HUẤN LUYỆN (Giữ nguyên logic cũ)
    X_grids_tensor = torch.tensor(np.array(FINAL_X_DL), dtype=torch.float32)
    X_phys_arr = np.array(FINAL_X_PHYS)
    y_arr = np.array(FINAL_Y)
    
    print(f"\n[PHẦN 2] Training Autoencoder & Ensemble trên {len(y_arr)} mẫu...")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ae_model = Conv1DAutoencoder().to(device)
    loader = DataLoader(TensorDataset(X_grids_tensor), batch_size=256, shuffle=True)
    optimizer = optim.Adam(ae_model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()
    
    for epoch in range(5):
        for batch in loader:
            inputs = batch[0].to(device)
            optimizer.zero_grad()
            out, _ = ae_model(inputs); loss = criterion(out, inputs)
            loss.backward(); optimizer.step()
            
    ae_model.eval()
    with torch.no_grad():
        X_dl_features = ae_model.get_latent(X_grids_tensor.to(device))

    ensemble = HybridEnsemble(scale_pos_weight=(np.sum(y_arr==0)/(np.sum(y_arr==1)+1e-6)))
    ensemble.fit(X_phys_arr, X_dl_features, y_arr)

    # PHẦN 4: TEST (Dùng Parallel im lặng tương tự)
    print(f"\n[PHẦN 3] Dự đoán Test (Dùng 100% CPU)...")
    ALL_TEST_DL, ALL_TEST_PHYS, ALL_TEST_IDS = [], [], []
    for split_name in split_dirs:
        T_LC_PATH = os.path.join(BASE_DIR, split_name, 'test_full_lightcurves.csv')
        if not os.path.exists(T_LC_PATH): continue
        print(f"  >> Đang chạy Split: {split_name}...")
        test_lc = test_processor.load_and_process(T_LC_PATH)
        test_lc.rename(columns={next(c for c in test_lc.columns if 'mjd' in c.lower() or 'time' in c.lower()): 'Time'}, inplace=True)
        
        p_inputs = [(oid, test_lc[test_lc['object_id']==oid].copy(), 0, 0) for oid in test_lc['object_id'].unique()]
        b_res = Parallel(n_jobs=-1)(delayed(process_single_object)(pi, pd, pt, pa, physics_extractor) for pi, pd, pt, pa in p_inputs)
        
        for r in b_res:
            if r:
                ALL_TEST_DL.append(r[0]['dl_feat']); ALL_TEST_PHYS.append(r[0]['phys_feat']); ALL_TEST_IDS.append(r[0]['obj_id'])
        del test_lc; gc.collect()

    # PHẦN 5: GHI SUBMISSION
    X_test_tensor = torch.tensor(np.array(ALL_TEST_DL), dtype=torch.float32)
    with torch.no_grad():
        X_test_latent = ae_model.get_latent(X_test_tensor.to(device))
    probs = ensemble.predict_proba(np.array(ALL_TEST_PHYS), X_test_latent)
    
    sub = pd.DataFrame({'object_id': ALL_TEST_IDS, 'target': probs})
    SAMPLE = os.path.join(BASE_DIR, 'sample_submission.csv')
    if os.path.exists(SAMPLE):
        sub = pd.read_csv(SAMPLE)[['object_id']].merge(sub, on='object_id', how='left').fillna(0)
    sub.to_csv('/kaggle/working/submission.csv', index=False)
    print("✅ HOÀN THÀNH!")

if __name__ == "__main__":
    main()